<img align="left" src="https://github.com/Db2-DTE-POC/CPDDVHOL4/blob/main/media/Assets&ArchHeader.jpeg?raw=true">

In [ ]:
!wget -O CPDDVRestClassV402.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/CPDDVRestClassV402.ipynb
%run CPDDVRestClassV402.ipynb

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
sc

In [ ]:
def loadCSV(file):
    return SQLContext(sc).read.csv(file, header='true', inferSchema = 'true')

In [ ]:
sparkloadtimer = Timer()

In [ ]:
%%capture result
%%time 
df_data_aircraft = loadCSV('/project_data/data_asset/aircraft.csv')

In [ ]:
sparkloadtimer.timeTotal()

In [ ]:
%%capture result
%%time 
df_data_ontime_11 = loadCSV('/project_data/data_asset/ONTIME2011.csv')

In [ ]:
sparkloadtimer.timeTotal()

In [ ]:
%%capture result
%%time 
df_data_ontime_12 = loadCSV('/project_data/data_asset/ONTIME2012.csv')

In [ ]:
sparkloadtimer.timeTotal()

In [ ]:
%%capture result
%%time 
df_data_ontime_13 = loadCSV('/project_data/data_asset/ONTIME2013.csv')

In [ ]:
sparkloadtimer.timeTotal()

In [ ]:
%%capture result
%%time 
df_data_ontime_14 = loadCSV('/project_data/data_asset/ONTIME2014.csv')

In [ ]:
sparkloadtimer.timeTotal()

In [ ]:
%%capture result
%%time 
df_data_ontime_15 = loadCSV('/project_data/data_asset/ONTIME2015.csv')

In [ ]:
sparkloadtimer.timeTotal()

In [ ]:
times = sparkloadtimer.getList()
for t in times:
    print(t)

In [ ]:
df_data_aircraft.show(5)
df_data_aircraft.printSchema()

In [ ]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

df_data_ontime = unionAll(df_data_ontime_11, df_data_ontime_12, df_data_ontime_13, df_data_ontime_14, df_data_ontime_15)

In [ ]:
%%time
df_data_ontime.printSchema()

In [ ]:
%%time
df_data_aircraft.count()

In [ ]:
df_data_aircraft.printSchema()

In [ ]:
%%time
df_data_ontime.count()

In [ ]:
%%time
df_data_ontime.createGlobalTempView("ontime")

In [ ]:
%%time
df_data_aircraft.createGlobalTempView("aircraft")

In [ ]:
%%capture result
%%time
sql = \
"""
SELECT TAILNUMBER, MANUFACTURER, MODEL, FLIGHTDATE, UNIQUECARRIER, FLIGHTNUM, ORIGINCITYNAME, DESTCITYNAME, DEPDELAY, ARRDELAY
  FROM global_temp.ontime OT, global_temp.aircraft AC 
  WHERE AC.TAILNUMBER = OT.TAILNUM
  AND AC.MANUFACTURER = 'Boeing' 
  AND AC.MODEL LIKE 'B737%'
  AND OT.ORIGINSTATE = 'NJ'
  AND OT.DESTSTATE = 'CA'
  AND OT.TAXIOUT > 30
  AND OT.DISTANCE > 2000
  AND OT.DEPDELAY > 180
  ORDER BY ARRDELAY
"""
spark.sql(sql).show()

In [ ]:
sqlsparktimer = Timer()
sqlsparktimer.timeTotal()

In [ ]:
sparkload = sparkloadtimer.getTotalTime()/1000
sparksql = sqlsparktimer.getTotalTime()/1000
print("Time to Load Data into Spark: " + str(sparkload) + " s")
print("Time to run SQL: " + str(sparksql) + " s")
print("Total Time: " + str(sparkload + sparksql) + " s")

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig = plt.figure(figsize=(20, 6))
ax = fig.add_axes([0,0,1,1])
runs = ['Time to Load Data','Time to run SQL','Total Time']
runtime = [sparkload, sparksql, sparkload+sparksql]
ax.bar(runs, runtime)
plt.ylabel("Time in Seconds (Lower is better)", fontsize=16)
plt.xlabel("Performance Run", fontsize=16)
plt.show()